In [1]:
import csv
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn import svm

df = pd.read_csv('coba-training.csv', delimiter=',', index_col = False, encoding = "ISO-8859-1" )

In [2]:
df.head()

,Type,Mentions,Date,Media,Sentiment,Influencer
0,rt,baca sejarah kenapa privatisasi bumn,12/1/2018 23:59,Twitter,-1,@wong_kito_ (wajah_syari)
1,rt,habib bahar enggan maaf presiden jokowi,12/1/2018 23:59,Twitter,-1,@aryadi_ringga (PAK RT 01 ÃÆÃâÃâÃÂ®Ã...
2,rt,part,12/1/2018 23:59,Twitter,-1,@johnbralink (johntan)
3,mention,desa hebat indonesia kuat jokowimembangundesa...,12/1/2018 23:59,Twitter,-1,@rico_fajarrr (rico fajar)
4,rt,terima kasih proyek mangkrak kelar sesi presi...,12/1/2018 23:59,Twitter,-1,@Fsilolongan (Roccuzzo)


In [3]:
# random untuk training data dan testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Mentions'], 
                                                    df['Sentiment'],
                                                    random_state=0)

In [15]:
print(len(X_train))

7873


In [5]:
print(len(y_train), len(y_test))

7873 2625


In [6]:
print('X_train first entry:\n\n', X_train.iloc[0])
print('\n\nX_train shape: ', X_train.shape)

X_train first entry:

  sandiaga uno undang jokowi adu data harga pangan


X_train shape:  (7873,)


# Feature selection TFIDF

In [10]:
# pembobotan setiap term yang ada pada document menggunakan rumus tf-idf
# naive bayes tidak bisa membaca fitur dalam bentuk kata, dia bisa membaca dalam bentuk angka
vect = TfidfVectorizer(min_df=5, use_idf=True, ngram_range=(1,2)).fit(X_train)
# vect = TfidfVectorizer(min_df=5, use_idf=True).fit(X_train)
# vect.get_feature_names()
print(vect.get_feature_names())
# tf = TfidfVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
# tfidf_matrix = tf.fit_transform(X_train)
# print(tfidf_matrix.toarray())
# tf

['aamiin', 'aamiin didukung', 'aamiin gagal', 'aamiin mengalir', 'abadi', 'abadi triliun', 'abang', 'abdul', 'abdul holik', 'abri', 'abri polri', 'acara', 'acara acara', 'acara bangsa', 'acara btn', 'acara dihadiri', 'acara jokowi', 'acara kongres', 'acara penyerahan', 'acara prabowo', 'acara presiden', 'acara reuni', 'acara sampah', 'aceh', 'aceh indah', 'adat', 'adat melayu', 'adat seri', 'adian', 'adian gaya', 'adlh', 'adlh jokowi', 'adu', 'adul', 'adul sekolah', 'advokat', 'advokat deklarasikan', 'agama', 'agen', 'agenda', 'agraria', 'agraria baik', 'ahmad', 'ahmad basarah', 'ahok', 'ahok dimusuhi', 'ahok kasar', 'ahok protes', 'air', 'air iman', 'ajak', 'ajak rakyat', 'aje', 'akar', 'akar rumput', 'akbar', 'akibat', 'aksi', 'aksi memanah', 'aktivis', 'akui', 'akui sebarkn', 'akun', 'akutu', 'akutu bocimijokowibikinhepi', 'al', 'ala', 'ala jokowi', 'alam', 'alam kebun', 'alasan', 'alfatihah', 'alfatihah rendahkan', 'alhamdulillah', 'aliran', 'allah', 'allah diumumkan', 'allah merdu

In [12]:
# Menghitung TF-IDF dari setiap kata yang terpilih diatas
X_train_vectorized = vect.transform(X_train)
print(X_train_vectorized)

  (0, 3264)	0.331204670702002
  (0, 3258)	0.28384753064078605
  (0, 3257)	0.2387501419312901
  (0, 2802)	0.331204670702002
  (0, 2801)	0.32133263072127316
  (0, 2249)	0.343931943815012
  (0, 1165)	0.36187002777244626
  (0, 1163)	0.07556272032531955
  (0, 974)	0.30242218325800346
  (0, 586)	0.31326658674456775
  (0, 32)	0.3086053576082632
  (1, 2654)	0.25343225333292163
  (1, 2650)	0.20636603730441153
  (1, 2428)	0.25376860143333013
  (1, 2425)	0.2483306206457512
  (1, 1764)	0.25276610239180886
  (1, 1335)	0.2524362433674435
  (1, 1334)	0.24624888386984997
  (1, 1179)	0.25276610239180886
  (1, 1163)	0.08760770151555651
  (1, 873)	0.2501847828202326
  (1, 872)	0.2483306206457512
  (1, 332)	0.25276610239180886
  (1, 331)	0.25276610239180886
  (1, 292)	0.25376860143333013
  :	:
  (7871, 3306)	0.2898053837085961
  (7871, 2593)	0.3019913618007861
  (7871, 2557)	0.13485028690515785
  (7871, 2545)	0.10493355359291928
  (7871, 2391)	0.30710108565117616
  (7871, 1558)	0.30710108565117616
  (7871

# Classification by Naive Bayes Classifier

In [17]:
mnb = MultinomialNB()
# men-training data hasil selection fitur kedalam fungsi Multinomial bayes
mnb.fit(X_train_vectorized, y_train)
print(X_train_vectorized, y_train)
# predictions = mnb.predict(vect.transform(X_test))

# print('Confusion Metrix: ', confusion_matrix(y_test,predictions))
# print('ACC: ', accuracy_score(y_test, predictions))

  (0, 3264)	0.331204670702002
  (0, 3258)	0.28384753064078605
  (0, 3257)	0.2387501419312901
  (0, 2802)	0.331204670702002
  (0, 2801)	0.32133263072127316
  (0, 2249)	0.343931943815012
  (0, 1165)	0.36187002777244626
  (0, 1163)	0.07556272032531955
  (0, 974)	0.30242218325800346
  (0, 586)	0.31326658674456775
  (0, 32)	0.3086053576082632
  (1, 2654)	0.25343225333292163
  (1, 2650)	0.20636603730441153
  (1, 2428)	0.25376860143333013
  (1, 2425)	0.2483306206457512
  (1, 1764)	0.25276610239180886
  (1, 1335)	0.2524362433674435
  (1, 1334)	0.24624888386984997
  (1, 1179)	0.25276610239180886
  (1, 1163)	0.08760770151555651
  (1, 873)	0.2501847828202326
  (1, 872)	0.2483306206457512
  (1, 332)	0.25276610239180886
  (1, 331)	0.25276610239180886
  (1, 292)	0.25376860143333013
  :	:
  (7871, 3306)	0.2898053837085961
  (7871, 2593)	0.3019913618007861
  (7871, 2557)	0.13485028690515785
  (7871, 2545)	0.10493355359291928
  (7871, 2391)	0.30710108565117616
  (7871, 1558)	0.30710108565117616
  (7871

In [48]:
# feature_names = np.array(vect.get_feature_names())

# sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

# print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
# print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

In [49]:
dftest = pd.read_csv('Jokowi-mentah-elastic.csv', delimiter=',', index_col = False, encoding = "ISO-8859-1" )

In [50]:
# result = dftest[:0]
# result.head(50)
# check = []

In [51]:
# i=1
# temp = []
# for row in dftest.iterrows():
#     temp.insert(len(temp),row)
#     if(i==500):
#         for x in temp:
#             hasil = mnb.predict(vect.transform([x['Mentions']]))
#             x['sentiment'] = dftest.apply (lambda x: hasil,axis=1)
#         check.insert(len(check),row['sentiment'])
#         temp = []
#         i=1
#         print("Checked")
         

In [52]:
# print(check)

In [53]:
# dftest = dftest[:500]

In [54]:
hasil = []
for index, row in dftest.iterrows():
    temp = mnb.predict(vect.transform([row['Mentions']]))
    hasil.append(temp[0])
#     row['sentiment'] = dftest.apply (lambda x: hasil,axis=1)
#     dftest['sentiment'] = dftest['sentiment'].replace({'[1]': 'Positive', '[0]': 'Neutral', '[-1]': 'Negative'})
#     dftest.to_csv("coba-out.csv", index=False, encoding = "ISO-8859-1")
    print(temp[0])

-1
0
0
1
0
-1
0
1
0
1
1
1
0
-1
0
0
0
0
1
0
-1
0
1
0
0
0
-1
0
-1
0
-1
0
-1
-1
0
0
-1
-1
-1
0
0
0
0
-1
0
0
-1
-1
0
-1
-1
0
0
0
-1
-1
-1
1
1
-1
-1
-1
0
-1
-1
0
1
0
0
0
0
0
0
-1
0
-1
1
0
0
-1
0
-1
1
0
0
0
-1
1
0
0
0
0
-1
-1
0
0
1
-1
1
0
0
-1
0
-1
0
0
-1
1
0
1
-1
-1
-1
0
0
1
1
0
0
0
0
-1
0
1
1
0
0
-1
0
0
1
-1
0
1
0
0
0
0
0
0
0
-1
-1
0
0
0
1
0
0
0
1
0
0
-1
-1
0
1
0
0
0
0
0
0
0
0
0
0
-1
0
-1
0
0
0
-1
0
0
1
0
1
0
0
1
0
0
-1
0
0
-1
-1
1
1
0
0
0
0
-1
-1
1
1
0
0
0
-1
0
0
1
0
0
0
-1
0
1
1
0
0
0
1
0
-1
-1
-1
1
1
0
-1
1
-1
0
0
1
-1
1
0
-1
0
1
0
-1
0
0
1
-1
1
0
0
0
0
-1
1
0
-1
0
0
-1
-1
0
-1
-1
0
0
-1
0
0
0
-1
0
0
1
0
0
0
0
0
0
0
0
-1
0
0
0
0
-1
1
0
1
0
0
0
1
-1
0
0
0
0
-1
1
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
1
0
0
-1
0
0
0
1
0
0
-1
0
0
0
0
0
-1
0
-1
0
0
0
1
0
0
0
-1
0
0
0
0
0
0
0
1
0
-1
0
0
0
0
-1
1
-1
0
0
-1
0
0
0
0
1
0
0
0
-1
1
0
0
-1
0
0
0
0
0
-1
1
0
1
0
-1
0
0
-1
0
-1
-1
1
-1
-1
1
-1
-1
0
0
0
-1
0
1
-1
0
-1
0
-1
-1
0
0
0
1
1
0
1
1
0
-1
0
1
0
-1
0
0
-1
-1
0
0
0
0
0
0
0
0
0
0
0
-1
-1
-1
1
0
1
-1
0
1
0
0

KeyboardInterrupt: 

In [92]:
print(dftest.shape)
print(len(hasil))
dftest['Sentiment'] = hasil
dftest['Sentiment'] = dftest['Sentiment'].replace({1: 'Positive', 0: 'Neutral', -1: 'Negative'})
dftest.to_csv("coba-out.csv", index=False, encoding = "ISO-8859-1")

(150000, 5)
150000


In [ ]:
dftest['sentimet'] = hasil